In [1]:
using Pkg

Pkg.activate(".")

  Activating project at `~/Documents/Uni/Master/MA/visual_analysis`


In [97]:
using GLMakie
using GeoMakie
using EmpiricalOrthogonalFunctions
using NCDatasets 
using Dates
using BenchmarkTools
using Statistics

In [163]:
include("utils.jl")

get_sliding_time_scopes_by_threshold (generic function with 2 methods)

In [4]:
base_path = "/home/denis/workspace/data/ivt_fields_v1"

"/home/denis/workspace/data/ivt_fields_v1"

In [5]:
winter_months = [12, 1, 2, 3]

tl_data = filter_by_date(build_timeline_data(base_path, 10, "ssp126", "ssp585"; file_range_selection = 4:5)) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2075-01-01T06:00:00"), DateTime("2075-01-01T12:00:00"), DateTime("2075-01-01T18:00:00"), DateTime("2075-01-02T00:00:00"), DateTime("2075-01-02T06:00:00"), DateTime("2075-01-02T12:00:00"), DateTime("2075-01-02T18:00:00"), DateTime("2075-01-03T00:00:00"), DateTime("2075-01-03T06:00:00"), DateTime("2075-01-03T12:00:00")  …  DateTime("2100-12-29T18:00:

In [93]:

# create geographical axes
function local_geoaxis_creation!(
    figure::Makie.Figure,
    lonlims::Tuple{<:Number, <:Number}, 
    latlims::Tuple{<:Number, <:Number}; 
    figure_row::Int = 1,
    figure_col::Int = 1,
    title = "",
    source_projection = "+proj=longlat +datum=WGS84",
    dest_projection = "+proj=merc",
    )

    geoaxis = GeoMakie.GeoAxis(
        figure[figure_row, figure_col]; 
        dest = dest_projection,
        source = source_projection,
        limits=(lonlims, latlims),
        title = title
    )

    return geoaxis
end


function build_figure(
    data,
    lon_bounds::Tuple{<:Number, <:Number},
    lat_bounds::Tuple{<:Number, <:Number};
    title::String = "",
    colormap = :viridis,
    colorrange::Union{Nothing, Tuple{<:Real, <:Real}} = nothing, 
    shading = Makie.automatic,
    resolution::Union{Nothing, Tuple{Int, Int}} = nothing
    )::Makie.Figure

    fig = isnothing(resolution) ?  Figure(fontsize=12) : Figure(size = resolution, fontsize=12)

    minval, maxval = extrema(data)

    ga = local_geoaxis_creation!(fig, lon_bounds, lat_bounds; title=title)
    surface!(ga, lon_bounds[1]..lon_bounds[2], lat_bounds[1]..lat_bounds[2], data; shading = shading, colormap = colormap)
    lines!(ga, GeoMakie.coastlines(); color = :white, transformation = (; translation = (0, 0, 1000)))
    Colorbar(fig[1, 2], limits = (minval, maxval), colormap = colormap)
    return fig
end

build_figure (generic function with 1 method)

In [67]:
function get_index_last_winter_day(time_data, starting_point, n, threshhold = Day(100))
    result = -1
    for i in starting_point:length(time_data) - 1
        if time_data[i+1] - time_data[i] > threshhold
            result = i
            break
        end
    end

    next_iter = n - 1

    if next_iter < 1 || result == -1
        return result
    else
        return get_index_last_winter_day(time_data, result + 1, next_iter, threshhold)
    end
end

get_index_last_winter_day (generic function with 2 methods)

In [142]:
function get_time_scopes_by_threshold(time_data, n, threshold = Day(100))
    result = []

    all_starting_points = [1]

    for i in 1:length(time_data)-1 
        if time_data[i+1] - time_data[i] > threshold
            push!(all_starting_points, i+1)
        end
    end

    function get_next_stop(start)

        counter = 1

        for i in start:length(time_data)-1 
            if time_data[i+1] - time_data[i] > threshold
                if counter == n
                    return i
                else
                    counter += 1
                end
            end
        end

        return 
    end
    
    

    return [sp:get_next_stop(sp) for sp in all_starting_points if !isnothing(get_next_stop(sp))]
end

get_time_scopes_by_threshold (generic function with 2 methods)

In [141]:
get_scopes(tl_data.time, 4)


23-element Vector{UnitRange{Int64}}:
 1:1815
 360:2299
 848:2787
 1332:3271
 1816:3755
 2300:4239
 2788:4727
 3272:5211
 3756:5695
 4240:6179
 ⋮
 6668:8607
 7152:9091
 7636:9575
 8120:10059
 8608:10547
 9092:11031
 9576:11515
 10060:11999
 10548:12483

In [ ]:
struct EOFMode
    spatial_data::AbstractArray{<: AbstractFloat, 3}
end

struct EOFResult
    
end

In [ ]:
function sliding_eof_analysis(timeline_data, season_number, modes)
    
end

In [70]:
first_winter = tl_data.time[1:get_index_last_winter_day(tl_data.time, 1, 4)]
first_few_winter_ivt = tl_data.datasets[1].data[:, :, 1:get_index_last_winter_day(tl_data.time, 1, 4)]
# get_index_last_winter_day(tl_data.time, 1, 4)
# tl_data.time[1:720]

70×32×1815 Array{Union{Missing, AbstractFloat}, 3}:
[:, :, 1] =
 194.783    234.968    315.58     487.98    …   9.61129   6.77699   3.93124
 268.127    278.475    328.706    453.229      10.7007    7.85825   4.16744
 234.717    236.177    282.117    404.252      11.3266    8.66736   4.45234
 159.283    160.484    208.555    342.637      11.6676    9.2254    4.74322
 119.699    111.413    153.835    293.258      11.7504    9.55738   4.91146
  94.2566    76.3358   130.89     280.497   …  11.7791    9.68234   4.83508
  71.6673    47.138    122.926    281.705      11.9618    9.53868   4.89562
  56.3926    43.3388   117.417    283.007      12.2251    9.20101   4.78637
  43.3779    46.6062   118.976    297.114      12.4885    8.62171   4.67754
  41.098     49.1267   143.253    324.798      12.6826    7.79042   4.63088
   ⋮                                        ⋱             ⋮        
   2.71991   11.0745    32.0793    76.6183     50.2556   39.5134   31.3674
   6.82419    6.93176   28.8749  

In [85]:
timesignal, spatial_signal, eigenvals = get_eof_of_datachunk(first_few_winter_ivt)

([-1890.781536991958 726.7162540103675 … 0.004616125497734181 8.930752247312824e-13; -1758.3095791587043 443.0983732055988 … -0.032462380525578906 8.930752247314022e-13; … ; 1295.4250640892812 2015.5804955048632 … -0.14416530570602787 8.930752247324188e-13; 1055.4206410091365 2263.9064039015675 … -0.052810634630119625 8.93075224733352e-13], Union{Missing, Float64}[0.004557728439350539 -0.0003600015485457147 … -0.0008641479569932789 -0.0006459608954014677; 0.004702749207050161 -0.0011444440782925553 … -0.000865619030256713 -0.000656007585708618; … ; 0.004407901854520837 0.004448106787751211 … 0.0019920079061062157 0.0015199629254151047; 0.006967578040040083 0.006167289153308582 … 0.0017513650969696593 0.0013822938728173587;;; -0.01109622380689218 -0.014766390838441295 … -1.9128472090983703e-5 -7.17638574416042e-5; -0.013769141483334496 -0.017342072590545706 … 4.9968660905919706e-5 7.537614492208425e-6; … ; -0.004413088218920758 -0.006338815670537992 … 0.0010360957362062287 0.00050846068

In [88]:
percentage_eigenvals = eigenvals ./ sum(eigenvals) 

1815-element Vector{Float64}:
 0.11969623911553393
 0.08430684471395829
 0.07527810169259362
 0.05289135313377677
 0.04144927203384052
 0.037949744259266734
 0.03443217967201929
 0.028593118652845077
 0.025853344855105095
 0.02485655942989921
 ⋮
 2.5277871626954513e-9
 2.411672272031069e-9
 2.393070474724768e-9
 2.331154215022325e-9
 2.229976316190545e-9
 2.025703386611351e-9
 1.8508199040406412e-9
 1.8204984566400077e-9
 2.3629006614614115e-32

In [94]:
build_figure(spatial_signal[:, :, 1], (-90, 40), (20, 80); title = "IVT Mode 1", colormap = :managua100)

In [20]:

function animate_timeline(
    data::TimelineData,
    filename::String;
    framerate::Int = 30,
    colormap = :viridis,
    coastline_color = :white,
    colorrange::Union{Nothing, Tuple{<:Real, <:Real}} = nothing, 
    shading = Makie.automatic,
    resolution::Union{Nothing, Tuple{Int, Int}} = nothing
    )::Makie.Figure

    fig = isnothing(resolution) ?  Figure(fontsize=12) : Figure(size = resolution, fontsize=12)

    time_index = Observable(1)

    axis = Dict()

    lon_bounds = extrema(data.lons)
    lat_bounds = extrema(data.lats)

    all_extrema = extrema.([d.data for d in data.datasets])

    min_val, max_val = reduce((a, b) -> (min(a[1], b[1]), max(a[2], b[2])), all_extrema)
    Colorbar(fig[2, 1:2], limits = (min_val, max_val), colormap = colormap, vertical = false,  label = "IVT (norm) in kg s^-1 m^-1")

    for (i, dataset) in enumerate(data.datasets)
        axis[dataset.name] = local_geoaxis_creation!(fig, lon_bounds, lat_bounds; title=@lift("$(dataset.name) $(Dates.format(data.time[$time_index], "mm/yyyy"))"), figure_col=i)
    end
    
    for dataset in data.datasets

        array_slice = @lift(dataset.data[:, :, $time_index])
        
        surface!(axis[dataset.name], lon_bounds[1]..lon_bounds[2], lat_bounds[1]..lat_bounds[2], array_slice; shading = shading, colormap = colormap, colorrange = (min_val, max_val))
        lines!(axis[dataset.name], GeoMakie.coastlines(); color = coastline_color, transformation = (; translation = (0, 0, 1000)))
    end

    timestamps = eachindex(data.time)
    record(fig, filename, timestamps; framerate = framerate) do t
        time_index[] = t
    end

    
    return fig
end

animate_timeline (generic function with 1 method)

In [23]:
# animate_timeline(tline_data, "scenario_comparison_2.mp4"; colormap = :managua100, resolution = (1000, 1000))
animate_timeline(tl_data, "scenario_comparison_end.mp4"; colormap = :managua100)

In [160]:
include("visual_utils.jl")

show_eof_modes_of_timeline (generic function with 1 method)

In [167]:
get_sliding_time_scopes_by_threshold(tl_data.time, 3)

24-element Vector{UnitRange{Int64}}:
 1:1331
 360:1815
 848:2299
 1332:2787
 1816:3271
 2300:3755
 2788:4239
 3272:4727
 3756:5211
 4240:5695
 ⋮
 7152:8607
 7636:9091
 8120:9575
 8608:10059
 9092:10547
 9576:11031
 10060:11515
 10548:11999
 11032:12483

In [169]:
show_eof_modes_of_timeline(tl_data, 4, 4, "eof_modes.mp4"; colormap = :managua100, framerate = 1)

First scope: 1:1815
